# AV-HuBERT Data Preprocessing

This notebook prepares audio‑visual speech data from TCD-Timit **for AV‑HuBERT finetuning**.  
It follows the following plan

- **Step 1:** Download/acquire dataset (change a few var's to point where you want to download) 
- **Step 2:** Convert **video → 25 fps**, **audio → 16 kHz mono**, then **crop mouth ROI** from detect_landmarks.py 
- **(Optional)** Data augmentation - this is part of my project but can be used without this  
- **Step 3:** Build AV‑HuBERT **manifests** (`train.tsv`, `valid.tsv`, `test.tsv`, `.wrd`, `.ltr`) and **vocabulary** (SentencePiece).

> Notes
> - Works **both in Colab and locally**.

In [60]:
# %% Download TCD-TIMIT volunteer bundle(s) for local Windows/macOS/Linux
# - Handles Windows paths safely
# - Resumes partial downloads
# - Verifies ZIP integrity

# Make Requests/urllib3 use Windows’ built-in certificate store
import truststore
truststore.inject_into_ssl()


import os, zipfile, shutil, pathlib, http.cookiejar, requests
from tqdm.auto import tqdm

# --- EDIT THESE THREE ONLY ---
VOL = "04M"  # e.g., "01M", "02M", etc.
DATA_ROOT = pathlib.Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
COOKIES_PATH = pathlib.Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\data_processing\cookies.txt")  # Netscape format; leave as non-existent if not needed
# -----------------------------

BASE_URL = f"https://sigmedia.tcd.ie/tcd_timit_db/volunteers/{VOL}/Clips"
OUT_DIR  = DATA_ROOT / "volunteers" / VOL / "Clips"
TMP_DIR  = DATA_ROOT / "_tmp" / "volunteers" / VOL / "Clips"
FILES    = ["30degcam.zip"]   # add/remove items as needed

OUT_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)

# Load cookies if present (optional)
cookies_jar = None
if COOKIES_PATH.exists():
    try:
        cj = http.cookiejar.MozillaCookieJar()
        cj.load(str(COOKIES_PATH), ignore_discard=True, ignore_expires=True)
        cookies_jar = cj
        print(f"[info] Loaded cookies from {COOKIES_PATH}")
    except Exception as e:
        print(f"[warn] Failed to load cookies ({e}). Proceeding without cookies.")

def download_with_resume(url: str, out_path: pathlib.Path):
    headers = {"User-Agent": "Mozilla/5.0"}
    # Resume if partial file exists
    start = out_path.stat().st_size if out_path.exists() else 0
    if start > 0:
        headers["Range"] = f"bytes={start}-"
    with requests.get(url, headers=headers, stream=True, allow_redirects=True, cookies=cookies_jar) as r:
        if r.status_code not in (200, 206):
            raise RuntimeError(f"HTTP {r.status_code} for {url}")
        total = int(r.headers.get("content-length", 0)) + start
        mode = "ab" if start > 0 else "wb"
        with open(out_path, mode) as f, tqdm(total=total, initial=start, unit="B", unit_scale=True, desc=out_path.name) as pbar:
            for chunk in r.iter_content(chunk_size=1<<20):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

def verify_zip(zpath: pathlib.Path):
    with zipfile.ZipFile(zpath) as z:
        bad = z.testzip()
        if bad:
            raise RuntimeError(f"ZIP corrupt: first bad file {bad}")

downloaded = []
for fname in FILES:
    url = f"{BASE_URL}/{fname}"
    tmp = TMP_DIR / fname
    print(f"\n[download] {url}")
    download_with_resume(url, tmp)
    print("[verify] testing ZIP integrity…")
    verify_zip(tmp)
    tgt = OUT_DIR / fname
    shutil.copy2(tmp, tgt)
    downloaded.append(tgt)
    print(f"[saved] {tgt}")

print("\nDone. Archives in:", OUT_DIR)
for p in downloaded:
    print(" •", p)


[info] Loaded cookies from C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\data_processing\cookies.txt

[download] https://sigmedia.tcd.ie/tcd_timit_db/volunteers/04M/Clips/30degcam.zip


30degcam.zip: 4.29GB [07:45, 9.21MB/s]


[verify] testing ZIP integrity…
[saved] C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\04M\Clips\30degcam.zip

Done. Archives in: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\04M\Clips
 • C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\04M\Clips\30degcam.zip


In [64]:
# %% Download TCD-TIMIT bundles (volunteers OR lipspeakers)
# - Safe Windows paths
# - Resumable downloads
# - ZIP integrity check

# Use OS trust store on Windows
import truststore
truststore.inject_into_ssl()

import os, zipfile, shutil, pathlib, http.cookiejar, requests
from tqdm.auto import tqdm

# --- EDIT THESE ONLY ----------------------------------------------------------
GROUP        = "lipspeakers"             # "volunteers" or "lipspeakers"
PERSON       = "Lipspkr1"                    # e.g. "04M" (volunteers) or "Lipspkr1" (lipspeakers)
FILES        = ["straightcam.zip"]         # e.g. ["straightcam.zip","30degcam.zip","audio.zip","labels.zip"]
DATA_ROOT    = pathlib.Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
COOKIES_PATH = pathlib.Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\data_processing\cookies.txt")
# -----------------------------------------------------------------------------

# Resolve base URL and local folders from GROUP/PERSON
GROUP = GROUP.strip().lower()
if GROUP not in {"volunteers", "lipspeakers"}:
    raise ValueError("GROUP must be 'volunteers' or 'lipspeakers'")

BASE_URL = f"https://sigmedia.tcd.ie/tcd_timit_db/{GROUP}/{PERSON}/Clips"
OUT_DIR  = DATA_ROOT / GROUP / PERSON / "Clips"
TMP_DIR  = DATA_ROOT / "_tmp" / GROUP / PERSON / "Clips"

OUT_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)

# Optional cookies
cookies_jar = None
if COOKIES_PATH and COOKIES_PATH.exists():
    try:
        cj = http.cookiejar.MozillaCookieJar()
        cj.load(str(COOKIES_PATH), ignore_discard=True, ignore_expires=True)
        cookies_jar = cj
        print(f"[info] Loaded cookies from {COOKIES_PATH}")
    except Exception as e:
        print(f"[warn] Failed to load cookies ({e}). Proceeding without cookies.)")

def download_with_resume(url: str, out_path: pathlib.Path):
    headers = {"User-Agent": "Mozilla/5.0"}
    # Resume support
    start = out_path.stat().st_size if out_path.exists() else 0
    if start > 0:
        headers["Range"] = f"bytes={start}-"
    with requests.get(url, headers=headers, stream=True, allow_redirects=True, cookies=cookies_jar) as r:
        if r.status_code not in (200, 206):
            raise RuntimeError(f"HTTP {r.status_code} for {url}")
        total = int(r.headers.get("content-length", 0)) + start
        mode = "ab" if start > 0 else "wb"
        with open(out_path, mode) as f, tqdm(
            total=total, initial=start, unit="B", unit_scale=True, desc=out_path.name
        ) as pbar:
            for chunk in r.iter_content(chunk_size=1<<20):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

def verify_zip(zpath: pathlib.Path):
    with zipfile.ZipFile(zpath) as z:
        bad = z.testzip()
        if bad:
            raise RuntimeError(f"ZIP corrupt: first bad file {bad}")

downloaded = []
for fname in FILES:
    url = f"{BASE_URL}/{fname}"
    tmp = TMP_DIR / fname
    print(f"\n[download] {url}")
    download_with_resume(url, tmp)
    print("[verify] testing ZIP integrity…")
    verify_zip(tmp)
    tgt = OUT_DIR / fname
    shutil.copy2(tmp, tgt)
    downloaded.append(tgt)
    print(f"[saved] {tgt}")

print("\nDone. Archives in:", OUT_DIR)
for p in downloaded:
    print(" •", p)


[info] Loaded cookies from C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\data_processing\cookies.txt

[download] https://sigmedia.tcd.ie/tcd_timit_db/lipspeakers/Lipspkr1/Clips/straightcam.zip


straightcam.zip: 31.9GB [47:45, 10.2MB/s]                    


[verify] testing ZIP integrity…
[saved] C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\straightcam.zip

Done. Archives in: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips
 • C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\straightcam.zip


In [67]:
import time, zipfile, requests
from pathlib import Path
from requests.exceptions import ChunkedEncodingError, ConnectionError, Timeout, RequestException

RETRY_MAX    = 12
RETRY_SLEEP  = 30
CHUNK        = 8 * (1<<20)   # 8 MiB
TIMEOUT      = (15, 120)     # (connect, read)
TRUNCATE_N   = 8 * (1<<20)   # trim 8 MiB on stubborn 416

def _zip_ok(p: Path) -> bool:
    try:
        with zipfile.ZipFile(p) as z:
            return z.testzip() is None
    except zipfile.BadZipFile:
        return False

def _truncate_tail(p: Path, nbytes: int):
    if not p.exists(): return
    size = p.stat().st_size
    with open(p, "r+b") as f:
        f.truncate(max(0, size - nbytes))

def download_with_resume(url: str, out_path: Path, cookies_jar=None):
    out_path.parent.mkdir(parents=True, exist_ok=True)

    for attempt in range(1, RETRY_MAX+1):
        have = out_path.stat().st_size if out_path.exists() else 0
        headers = {"User-Agent": "Mozilla/5.0"}
        if have > 0:
            headers["Range"] = f"bytes={have}-"

        print(f"[try {attempt}] GET {url} (resume from {have:,} bytes)")
        try:
            r = requests.get(
                url, headers=headers, stream=True, allow_redirects=True,
                timeout=TIMEOUT, cookies=cookies_jar
            )

            # If server returns 416, verify or truncate and retry
            if r.status_code == 416:
                print("[info] HTTP 416: checking existing ZIP on disk…")
                if _zip_ok(out_path):
                    print("[verify] ZIP OK — download complete.")
                    return
                print("[info] ZIP not valid yet; truncating tail and resuming…")
                _truncate_tail(out_path, TRUNCATE_N)
                time.sleep(RETRY_SLEEP)
                continue

            if r.status_code not in (200, 206):
                raise RuntimeError(f"HTTP {r.status_code}")

            # If we asked for a Range but got 200 (full file), start over to avoid duplicate content
            mode = "wb" if (have > 0 and r.status_code == 200) else ("ab" if have > 0 else "wb")
            start = 0 if mode == "wb" else have
            total = start + int(r.headers.get("content-length", 0)) if r.headers.get("content-length") else None

            with open(out_path, mode) as f:
                downloaded = start
                for chunk in r.iter_content(chunk_size=CHUNK):
                    if not chunk:
                        continue
                    f.write(chunk)
                    downloaded += len(chunk)

            # Verify after each attempt
            if _zip_ok(out_path):
                print("[verify] ZIP OK")
                return
            else:
                print("[verify] ZIP incomplete/corrupt — will resume.")
                time.sleep(RETRY_SLEEP)

        except (ChunkedEncodingError, ConnectionError, Timeout, RequestException, RuntimeError) as e:
            print(f"[warn] Transfer issue: {e}")
            time.sleep(RETRY_SLEEP)

    raise RuntimeError(f"Failed to obtain a valid ZIP after {RETRY_MAX} attempts")


In [68]:
downloaded = []
for fname in FILES:
    url = f"{BASE_URL}/{fname}"
    tmp = TMP_DIR / fname
    print(f"\n[download] {url}")
    download_with_resume(url, tmp)
    print("[verify] testing ZIP integrity…")
    verify_zip(tmp)
    tgt = OUT_DIR / fname
    shutil.copy2(tmp, tgt)
    downloaded.append(tgt)
    print(f"[saved] {tgt}")

print("\nDone. Archives in:", OUT_DIR)
for p in downloaded:
    print(" •", p)


[download] https://sigmedia.tcd.ie/tcd_timit_db/lipspeakers/Lipspkr1/Clips/straightcam.zip
[try 1] GET https://sigmedia.tcd.ie/tcd_timit_db/lipspeakers/Lipspkr1/Clips/straightcam.zip (resume from 31,865,357,204 bytes)
[info] HTTP 416: checking existing ZIP on disk…
[verify] ZIP OK — download complete.
[verify] testing ZIP integrity…
[saved] C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\straightcam.zip

Done. Archives in: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips
 • C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\straightcam.zip


In [69]:
import os, io, time, shutil, zipfile
from pathlib import Path
from tqdm.auto import tqdm

def unzip_with_progress(
    zpath: Path,
    dest: Path,
    *,
    skip_existing: bool = True,
    verify_zip: bool = True,
    strip_top_dir: bool = False,   # if the zip has a single top-level folder, drop it
    fail_if_low_space: bool = True,
    space_safety_margin: float = 1.10,  # require 10% headroom above uncompressed size
) -> list[Path]:
    """
    Extracts `zpath` into `dest` safely, with progress and optional skipping of existing files.
    Returns a list of extracted file paths.
    """

    zpath = Path(zpath)
    dest  = Path(dest)
    if not zpath.exists():
        raise FileNotFoundError(zpath)
    dest.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(zpath) as z:
        if verify_zip:
            bad = z.testzip()
            if bad:
                raise RuntimeError(f"ZIP is corrupt; first bad file: {bad}")

        infos = [m for m in z.infolist() if not m.is_dir()]
        if not infos:
            return []

        # Optionally strip a single top-level directory
        if strip_top_dir:
            parts = [i.filename.split('/', 1)[0] for i in infos]
            top_candidates = {p for p in parts if p}
            if len(top_candidates) == 1:
                top = next(iter(top_candidates)) + '/'
                for i in infos:
                    if i.filename.startswith(top):
                        i._filename = i.filename[len(top):]  # type: ignore[attr-defined]
                # fall back to original name if not set
                def _name(i): return getattr(i, "_filename", i.filename)
            else:
                def _name(i): return i.filename
        else:
            def _name(i): return i.filename

        # Disk space check (rough)
        total_uncompressed = sum(i.file_size for i in infos)
        free = shutil.disk_usage(dest).free
        if fail_if_low_space and total_uncompressed * space_safety_margin > free:
            raise RuntimeError(
                f"Not enough free space: need ~{total_uncompressed/1e9:.2f} GB "
                f"(+margin), have {free/1e9:.2f} GB"
            )

        # Helper: avoid zip-slip
        dest_real = dest.resolve()
        def _safe_target(name: str) -> Path:
            # normalize separators and drop leading slashes
            name = name.replace("\\", "/").lstrip("/")
            target = (dest_real / name).resolve()
            if not str(target).startswith(str(dest_real)):
                raise RuntimeError(f"Blocked unsafe path in ZIP: {name}")
            target.parent.mkdir(parents=True, exist_ok=True)
            return target

        extracted = []
        pbar = tqdm(total=total_uncompressed, unit="B", unit_scale=True, desc=f"Extracting {zpath.name}")
        for info in infos:
            name = _name(info)
            if not name or name.endswith("/"):
                continue
            tgt = _safe_target(name)

            if skip_existing and tgt.exists() and tgt.stat().st_size == info.file_size:
                pbar.update(info.file_size)
                continue

            # Stream copy to show incremental progress
            with z.open(info) as src, open(tgt, "wb") as dst:
                for chunk in iter(lambda: src.read(1<<20), b""):
                    dst.write(chunk)
                    pbar.update(len(chunk))

            # Preserve modified time if available
            try:
                ts = time.mktime((*info.date_time, 0, 0, -1))
                os.utime(tgt, (ts, ts))
            except Exception:
                pass

            extracted.append(tgt)

        pbar.close()
        return extracted


In [70]:
zip_file = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\straightcam.zip")
out_dir  = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips")

files = unzip_with_progress(
    zip_file, out_dir,
    skip_existing=True,
    verify_zip=True,
    strip_top_dir=False
)
print(f"Extracted {len(files)} files.")

Extracting straightcam.zip: 100%|██████████| 29.5G/29.5G [09:36<00:00, 51.2MB/s]  


Extracted 1509 files.


In [2]:
# Split an unzipped camera-angle folder into video/audio/landmarks/transcripts
# and place them under: TCD_TIMIT/volunteers/<VOL>/Clips/{video,audio,landmarks,transcripts}/<ANGLE>

import os, shutil
from pathlib import Path

# ---- EDIT THESE THREE ONLY ----
GROUP = "lipspeakers"
VOL   = "Lipspkr1"
ANGLE = "straightcam"   # e.g., "straightcam" or "30degcam"
# Path where you unzipped the camera angle (i.e., where the .mp4/.wav/.mat/.txt are now)
PATH_TO_UNZIPPED = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
PATH_TO_UNZIPPED = PATH_TO_UNZIPPED / GROUP / VOL / "Clips" / ANGLE
# --------------------------------

DATA_ROOT = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
OUT_DIR   = DATA_ROOT / GROUP / VOL / "Clips"

SRC_DIR = PATH_TO_UNZIPPED
assert SRC_DIR.exists(), f"Source folder not found: {SRC_DIR}"

# Destinations (this is the “correct format” you wanted)
VIDEO_DIR       = OUT_DIR / "video"       / ANGLE
AUDIO_DIR       = OUT_DIR / "audio"       / ANGLE
LANDMARKS_DIR   = OUT_DIR / "landmarks"   / ANGLE  # .mat (raw landmark files)
TRANSCRIPTS_DIR = OUT_DIR / "transcripts" / ANGLE  # .txt

for d in [VIDEO_DIR, AUDIO_DIR, LANDMARKS_DIR, TRANSCRIPTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def safe_move(src: Path, dst_dir: Path, overwrite=False):
    dst = dst_dir / src.name
    if dst.exists():
        if overwrite:
            dst.unlink()
        else:
            # keep existing; place a numbered suffix
            stem, suf = src.stem, src.suffix
            k = 1
            while True:
                cand = dst_dir / f"{stem}__dup{k}{suf}"
                if not cand.exists():
                    dst = cand
                    break
                k += 1
    shutil.move(str(src), str(dst))
    return dst

moved = {"video":0, "audio":0, "landmarks":0, "transcripts":0}

# Move by extension
for p in SRC_DIR.glob("*.mp4"):
    safe_move(p, VIDEO_DIR);       moved["video"] += 1
for p in SRC_DIR.glob("*.wav"):
    safe_move(p, AUDIO_DIR);       moved["audio"] += 1
for p in SRC_DIR.glob("*.mat"):
    safe_move(p, LANDMARKS_DIR);   moved["landmarks"] += 1
for p in SRC_DIR.glob("*.txt"):
    safe_move(p, TRANSCRIPTS_DIR); moved["transcripts"] += 1

print(f"[{ANGLE}] moved: "
      f'{moved["video"]} video, '
      f'{moved["audio"]} audio, '
      f'{moved["landmarks"]} landmarks, '
      f'{moved["transcripts"]} transcripts.')

print("Video  ->", VIDEO_DIR)
print("Audio  ->", AUDIO_DIR)
print("LMKs   ->", LANDMARKS_DIR)
print("Texts  ->", TRANSCRIPTS_DIR)


[straightcam] moved: 0 video, 0 audio, 0 landmarks, 0 transcripts.
Video  -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\video\straightcam
Audio  -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\audio\straightcam
LMKs   -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\landmarks\straightcam
Texts  -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\transcripts\straightcam


## Edit this cell to point to what is being processed 

## 0. Runtime & dependencies

In [1]:
# Minimal deps for this notebook (if your venv is fresh)
import sys, subprocess, os, platform

def pipi(pkg): 
    try: __import__(pkg.split("==")[0])
    except Exception: subprocess.run([sys.executable, "-m", "pip", "install", pkg], check=True)

for pkg in ["numpy", "opencv-python", "tqdm", "scikit-video", "scipy", "soundfile", "pillow", "dlib"]:
    pipi(pkg)

# Make sure ffmpeg is on PATH (or set the folder below)
FFMPEG_DIR = r"C:\ffmpeg\bin"  # change if needed
os.environ["PATH"] = FFMPEG_DIR + os.pathsep + os.environ["PATH"]
print("ffmpeg in PATH:", any("ffmpeg" in p.lower() for p in os.environ["PATH"].split(os.pathsep)))
print("Python:", sys.executable)
print("Platform:", platform.platform())


ffmpeg in PATH: True
Python: c:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\.venv\Scripts\python.exe
Platform: Windows-11-10.0.26100-SP0


In [2]:

# Minimal deps + cross-platform FFmpeg check (no apt on Windows/macOS)
import sys, os, shutil, subprocess
from pathlib import Path
import subprocess, shutil

def sh(args):
    print(">>", " ".join(args) if isinstance(args, list) else args)
    subprocess.run(args, check=True, shell=isinstance(args, str))

def pip_install(pkg):
    sh([sys.executable, "-m", "pip", "install", "-q", pkg])

# --- Python deps ---
try:
    import cv2  # noqa
except Exception:
    pip_install("opencv-python-headless==4.8.*")

for mod, spec in [("sentencepiece","sentencepiece"),
                  ("numpy","numpy==1.26.*"),
                  ("scipy","scipy==1.11.*"),
                  ("tqdm","tqdm")]:
    try:
        __import__(mod)
    except Exception:
        pip_install(spec)

# --- FFmpeg presence check (no auto-install on Windows/macOS) ---
ffmpeg_path = shutil.which("ffmpeg")
ffprobe_path = shutil.which("ffprobe")

if ffmpeg_path and ffprobe_path:
    print(f"FFmpeg OK: {ffmpeg_path}")
else:
    if sys.platform.startswith("win"):
        raise SystemExit(
            "FFmpeg not found on PATH.\n"
            "Install it once via ONE of these options, then restart VS Code/Terminal:\n"
            "  • winget install -e --id Gyan.FFmpeg\n"
            "  • choco install ffmpeg    (if you use Chocolatey)\n"
            "  • scoop install ffmpeg    (if you use Scoop)\n"
            "  • conda install -c conda-forge ffmpeg  (if you use Conda)\n"
            "After install, ensure the folder containing ffmpeg.exe/ffplay.exe is in PATH."
        )
    elif sys.platform == "darwin":
        raise SystemExit(
            "FFmpeg not found. On macOS run:\n"
            "  brew install ffmpeg\n"
            "Then restart your terminal and rerun this cell."
        )
    else:
        # Linux: try apt-get if available, else instruct user
        if shutil.which("apt-get"):
            sh("sudo apt-get update -y && sudo apt-get install -y ffmpeg")
            print("FFmpeg installed via apt.")
        elif shutil.which("yum"):
            sh("sudo yum install -y ffmpeg")
            print("FFmpeg installed via yum.")
        else:
            raise SystemExit(
                "FFmpeg not found and no known package manager detected.\n"
                "Install via your distro’s package manager and retry."
            )

print("Environment ready.")


FFmpeg OK: C:\Users\irish\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0-full_build\bin\ffmpeg.EXE
Environment ready.


## 1. Config — paths and options

In [3]:
from pathlib import Path

# --- project roots ---
DATA_ROOT  = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
GROUP = "lipspeakers"
SPEAKER    = "Lipspkr1"                 # e.g. "01M","02M"
CAM_ANGLE  = "straightcam"         # or "30degcam"

# --- inputs (after your 30->25fps + 48k->16k conversions) ---
VID25_DIR   = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/video25/{CAM_ANGLE}"
AUDIO16_DIR = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/audio16k/{CAM_ANGLE}"
TEXT_DIR    = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/text/{CAM_ANGLE}"  # optional

# --- outputs ---
LMK_DIR   = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/landmarks/{CAM_ANGLE}"   # .pkl
ROI88_DIR = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/video/{CAM_ANGLE}"        # 88x88 mp4s
LMK_DIR.mkdir(parents=True, exist_ok=True)
ROI88_DIR.mkdir(parents=True, exist_ok=True)

# --- tools / models ---
AVH_REPO  = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\av_hubert\av_hubert")
DETECT_PY = AVH_REPO / r"avhubert\preparation\detect_landmark.py"

DLIB_CNN  = Path(r"C:\models\dlib\mmod_human_face_detector.dat")
DLIB_L68  = Path(r"C:\models\dlib\shape_predictor_68_face_landmarks.dat")
FFMPEG_DIR = Path(r"C:\Users\irish\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0-full_build\bin")


In [4]:
# Create a sitecustomize shim inside your active venv
import sys, site, pathlib

# Find site-packages for the current interpreter
sp = next(p for p in site.getsitepackages() if "site-packages" in p)
shim = pathlib.Path(sp) / "sitecustomize.py"
shim.write_text(
    "import numpy as _np\n"
    "if not hasattr(_np, 'float'): _np.float = float\n"
    "if not hasattr(_np, 'int'): _np.int = int\n"
    "if not hasattr(_np, 'bool'): _np.bool = bool\n"
)
print(f"[ok] wrote {shim}")

# (Optional) show where Python will import sitecustomize from
import importlib, importlib.util
spec = importlib.util.find_spec("sitecustomize")
print("sitecustomize found at:", spec.origin if spec else "NOT FOUND")


[ok] wrote c:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\.venv\Lib\site-packages\sitecustomize.py
sitecustomize found at: c:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\.venv\Lib\site-packages\sitecustomize.py


## 2. Utilities — media I/O & helpers

In [5]:
import subprocess, json, os
from pathlib import Path

def run(cmd: list | str) -> None:
    cmd_str = " ".join(cmd) if isinstance(cmd, list) else cmd
    print(">>", cmd_str)
    subprocess.run(cmd_str, shell=True, check=True)

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def convert_audio(in_wav: Path, out_wav: Path, sr: int = 16000):
    ensure_dir(out_wav.parent)
    run([
        "ffmpeg -y -v error",
        f'-i "{in_wav}"',
        "-ac 1",
        f"-ar {sr}",
        "-c:a pcm_s16le",
        f'"{out_wav}"'
    ])

def convert_video_fps(in_mp4: Path, out_mp4: Path, fps: int = 25):
    ensure_dir(out_mp4.parent)
    run([
        "ffmpeg -y -v error",
        f'-i "{in_mp4}"',
        f"-r {fps}",
        "-c:v libx264 -preset veryfast -crf 18 -pix_fmt yuv420p",
        f'"{out_mp4}"'
    ])

# Optional: quick probe utilities
def ffprobe_json(path: Path) -> dict:
    out = subprocess.check_output(
        f'ffprobe -v error -print_format json -show_format -show_streams "{path}"',
        shell=True
    ).decode("utf-8")
    return json.loads(out)

def count_video_frames(mp4_path: Path) -> int:
    info = ffprobe_json(mp4_path)
    for st in info.get("streams", []):
        if st.get("codec_type") == "video":
            nb = st.get("nb_frames")
            if nb and nb != "N/A":
                return int(nb)
    return 0


## 3. Convert media (30 fps/48 kHz → 25 fps/16 kHz) and crop ROI

In [9]:
import sys, os, subprocess
from pathlib import Path

# === Configure per run ===
DATA_ROOT  = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
GROUP = "lipspeakers"                     # "volunteers" or "lipspeakers"
SPEAKER    = "Lipspkr1"                           # e.g., "01M", "02M"
CAM_ANGLE  = "straightcam"                   # or "30degcam"
FFMPEG_DIR = Path(r"C:\Users\irish\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0-full_build\bin")

# AV-HuBERT detect script + dlib models
DETECT_PY  = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\av_hubert\av_hubert\avhubert\preparation\detect_landmark.py")
DLIB_CNN   = Path(r"C:\models\dlib\mmod_human_face_detector.dat")
DLIB_L68   = Path(r"C:\models\dlib\shape_predictor_68_face_landmarks.dat")

# Inputs (25fps full-face) and desired landmarks dir
VID25_DIR  = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/video25/{CAM_ANGLE}"
LMK_DIR    = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/landmarks/{CAM_ANGLE}"
LMK_DIR.mkdir(parents=True, exist_ok=True)

# 1) Tool checks
assert DETECT_PY.exists(), f"Missing: {DETECT_PY}"
assert (FFMPEG_DIR / "ffmpeg.exe").exists(), "ffmpeg.exe not found"
assert (FFMPEG_DIR / "ffprobe.exe").exists(), "ffprobe.exe not found"
assert DLIB_CNN.exists(), f"Missing: {DLIB_CNN}"
assert DLIB_L68.exists(), f"Missing: {DLIB_L68}"

# 2) Runtime deps needed by detect_landmark.py
for pkg in ("scikit-video", "tqdm", "opencv-python", "numpy<2.0"):  # numpy<2 avoids skvideo np.float issues
    try:
        __import__(pkg.replace("-", "_").split("<")[0])
    except Exception:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=True)

# 3) Collect videos
mp4s = sorted(VID25_DIR.rglob("*.mp4"))
print(f"Found {len(mp4s)} MP4s under {VID25_DIR}")
if mp4s:
    print("First:", mp4s[0])

# 4) Build manifest (relative to DATA_ROOT, strip .mp4)
rel_noext = [str(p.relative_to(DATA_ROOT).with_suffix("")).replace("\\","/") for p in mp4s]
MANIFEST_LIST = DATA_ROOT / "file.list"
MANIFEST_LIST.write_text("\n".join(rel_noext), encoding="utf-8")
print(f"[ok] wrote {MANIFEST_LIST} ({len(rel_noext)} lines)")
print("  e.g.", rel_noext[0] if rel_noext else "(none)")

# 5) Put ffmpeg on PATH for skvideo
os.environ["PATH"] = str(FFMPEG_DIR) + os.pathsep + os.environ.get("PATH","")
subprocess.run("ffmpeg -hide_banner -version", shell=True, check=True)
subprocess.run("ffprobe -hide_banner -version", shell=True, check=True)

# Expose for next cells
print("\nReady for landmark detection.")
print("Variables available: MANIFEST_LIST, LMK_DIR, DATA_ROOT, DETECT_PY, DLIB_CNN, DLIB_L68")


Found 377 MP4s under C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25\straightcam
First: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25\straightcam\sa1.mp4
[ok] wrote C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\file.list (377 lines)
  e.g. lipspeakers/Lipspkr1/Clips/processed/video25/straightcam/sa1

Ready for landmark detection.
Variables available: MANIFEST_LIST, LMK_DIR, DATA_ROOT, DETECT_PY, DLIB_CNN, DLIB_L68


## Convert Video and Audio

In [8]:
# --- Config & imports for the 25 fps / 16 kHz conversion ---
from pathlib import Path
import os, subprocess

# Where the dataset lives
DATA_ROOT = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")

# Who and which camera angles to process
GROUP    = "lipspeakers"               # "volunteers" or "lipspeakers"
SPEAKERS = ["Lipspkr1"]                     # add more later, e.g. ["01M","02M"]
CAMS     = ["straightcam", "30degcam"] # process one or both

# Target formats
TARGET_FPS = 25
TARGET_SR  = 16000

# (Optional) If ffmpeg isn’t on PATH, point this to your bin folder:
FFMPEG_DIR = Path(r"C:\Users\irish\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0-full_build\bin")
if (FFMPEG_DIR / "ffmpeg.exe").exists():
    os.environ["PATH"] = str(FFMPEG_DIR) + os.pathsep + os.environ.get("PATH", "")

In [76]:
import re
from pathlib import Path
import subprocess, os
import cv2
from tqdm import tqdm

AUDIO_SRC_FOR_CAM = {"30degcam": "straightcam"}  # 30degcam shares audio with straightcam {"30degcam": "straightcam"}
TARGET_FPS = 25
TARGET_SR  = 16000

def sh(cmd):
    print(">>", cmd)
    subprocess.run(cmd, shell=True, check=True)

def ensure(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def convert_video_25fps(src_mp4: Path, dst_mp4: Path, fps=25):
    ensure(dst_mp4.parent)
    sh(f'ffmpeg -y -v error -i "{src_mp4}" -r {fps} -c:v libx264 -preset veryfast -crf 18 -pix_fmt yuv420p "{dst_mp4}"')

def convert_wav_16k(src_wav: Path, dst_wav: Path, sr=16000):
    ensure(dst_wav.parent)
    sh(f'ffmpeg -y -v error -i "{src_wav}" -ac 1 -ar {sr} -c:a pcm_s16le "{dst_wav}"')

def list_files(root: Path, ext: str):
    return sorted(root.rglob(f"*.{ext.lower()}"))

def normalize_stem(stem: str) -> str:
    # collapse things like "si1011__dup1" -> "si1011"
    return re.sub(r"__dup\d+$", "", stem)

ok_vid = ok_wav = 0
for spk in SPEAKERS:
    for cam in CAMS:
        # --- videos: convert per-cam as usual ---
        raw_vid_dir = DATA_ROOT / f"{GROUP}/{spk}/Clips/video/{cam}"
        vid25_dir   = DATA_ROOT / f"{GROUP}/{spk}/Clips/processed/video25/{cam}"
        for v in list_files(raw_vid_dir, "mp4"):
            out = vid25_dir / v.name
            if not out.exists():
                convert_video_25fps(v, out, fps=TARGET_FPS)
            ok_vid += 1

        # --- audio: only process once for the source cam ---
        src_cam = AUDIO_SRC_FOR_CAM.get(cam, cam)
        if cam != src_cam:
            # skip conversion for alias cams to avoid duplicates entirely
            continue

        raw_aud_dir = DATA_ROOT / f"{GROUP}/{spk}/Clips/audio/{src_cam}"
        aud16_dir   = DATA_ROOT / f"{GROUP}/{spk}/Clips/processed/audio16k/{src_cam}"

        # De-dup on name pattern: keep one per normalized stem
        seen = set()
        for a in list_files(raw_aud_dir, "wav"):
            stem_norm = normalize_stem(a.stem)
            if stem_norm in seen:
                # we've already converted this utterance; skip dup variants
                continue
            seen.add(stem_norm)

            out = aud16_dir / f"{stem_norm}.wav"
            if not out.exists():
                convert_wav_16k(a, out, sr=TARGET_SR)
            ok_wav += 1

print(f"[done] converted: videos={ok_vid}, audios={ok_wav}")


>> ffmpeg -y -v error -i "C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\audio\straightcam\sa1.wav" -ac 1 -ar 16000 -c:a pcm_s16le "C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\audio16k\straightcam\sa1.wav"
>> ffmpeg -y -v error -i "C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\audio\straightcam\sa2.wav" -ac 1 -ar 16000 -c:a pcm_s16le "C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\audio16k\straightcam\sa2.wav"
>> ffmpeg -y -v error -i "C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\audio\straightcam\si1001.wav" -ac 1 -ar 16000 -c:a pcm_s16le "C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\audi

## Create Manifest for detect_landmark.py


In [10]:
# Build manifest from the 25fps videos you want to detect
manifest_lines = []
for spk in SPEAKERS:
    for cam in CAMS:
        for mp4 in (DATA_ROOT / f"{GROUP}/{spk}/Clips/processed/video25/{cam}").glob("*.mp4"):
            rel_noext = str(mp4.relative_to(DATA_ROOT).with_suffix("")).replace("\\","/")
            manifest_lines.append(rel_noext)

MANIFEST_LIST = DATA_ROOT / "file.list"
MANIFEST_LIST.write_text("\n".join(sorted(manifest_lines)), encoding="utf-8")
print(f"[ok] wrote {MANIFEST_LIST} ({len(manifest_lines)} lines)")


[ok] wrote C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\file.list (377 lines)


In [11]:
import sys, subprocess, os
from pathlib import Path

# --- your known paths ---
FFMPEG_PATH = r"C:\Users\irish\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0-full_build\bin\ffmpeg.exe"

for cam in CAMS:
    LMK_DIR = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/landmarks/{cam}"
    LMK_DIR.mkdir(parents=True, exist_ok=True)

    MANIFEST_CAM = DATA_ROOT / f"file_{cam}.list"

    # filter manifest lines belonging to this camera
    all_lines = MANIFEST_LIST.read_text().splitlines()
    cam_lines = [ln for ln in all_lines if f"/{cam}/" in ln]
    MANIFEST_CAM.write_text("\n".join(cam_lines), encoding="utf-8")
    print(f"[ok] Created {MANIFEST_CAM} ({len(cam_lines)} entries)")

    cmd = [
        sys.executable, str(DETECT_PY),
        "--root", str(DATA_ROOT),
        "--landmark", str(LMK_DIR),
        "--manifest", str(MANIFEST_CAM),
        "--cnn_detector", str(DLIB_CNN),
        "--face_predictor", str(DLIB_L68),
        "--ffmpeg", FFMPEG_PATH,
        "--rank", "0", "--nshard", "1"
    ]

    print(">>", " ".join(cmd))
    subprocess.run(cmd, check=True)
    print(f"[done] Landmarks saved to {LMK_DIR}")


[ok] Created C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\file_straightcam.list (377 entries)
>> c:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\.venv\Scripts\python.exe C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\av_hubert\av_hubert\avhubert\preparation\detect_landmark.py --root C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT --landmark C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\landmarks\straightcam --manifest C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\file_straightcam.list --cnn_detector C:\models\dlib\mmod_human_face_detector.dat --face_predictor C:\models\dlib\shape_predictor_68_face_landmarks.dat --ffmpeg C:\Users\irish\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0-full_build\bin\ffmpeg.exe --rank 0 --nshard

In [6]:
import pickle, cv2, numpy as np
from pathlib import Path
from tqdm import tqdm

# ---- configure per run ----
DATA_ROOT  = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
GROUP = "lipspeakers"               # "volunteers" or "lipspeakers"
SPEAKER    = "Lipspkr1"                 # e.g., "01M", "02M"
CAM_ANGLE  = "straightcam"         # or "30degcam"
TARGET_SZ  = 88
MARGIN     = 1.6                   # enlarge mouth box (~60%)
SMOOTH_WIN = 7                     # frames for moving average

# Inputs/Outputs
VID25_DIR  = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/video25/{CAM_ANGLE}"
LMK_DIR    = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/landmarks/{CAM_ANGLE}"
OUT_DIR    = DATA_ROOT / f"{GROUP}/{SPEAKER}/Clips/processed/video25crop/{CAM_ANGLE}"
OUT_DIR.mkdir(parents=True, exist_ok=True)

MOUTH = list(range(48, 68))  # dlib 68-pt mouth indices

def tighten_xywh_from_mouth(pts):
    """pts: (68,2). Return [x,y,w,h] around mouth with margin."""
    mouth = np.asarray(pts)[MOUTH]  # (20,2)
    x0, y0 = mouth.min(axis=0)
    x1, y1 = mouth.max(axis=0)
    cx, cy = (x0+x1)/2, (y0+y1)/2
    side = max(x1-x0, y1-y0) * MARGIN
    return np.array([cx - side/2, cy - side/2, side, side], dtype=float)

def smooth_xywh(seq, win=7):
    """NaN-aware moving average over (T,4)."""
    seq = np.asarray(seq, float)
    # forward/backward fill NaNs
    for c in range(4):
        col = seq[:, c]
        nans = np.isnan(col)
        if nans.any():
            # forward fill
            idx = np.where(~nans, np.arange(len(col)), 0)
            np.maximum.accumulate(idx, out=idx)
            col[nans] = col[idx[nans]]
            # backward fill
            idx2 = np.where(~nans, np.arange(len(col)), len(col)-1)
            idx2 = np.minimum.accumulate(idx2[::-1])[::-1]
            col[nans] = col[idx2[nans]]
        seq[:, c] = col
    # box filter
    pad = (win-1)//2
    padseq = np.pad(seq, ((pad,pad),(0,0)), mode='edge')
    kernel = np.ones((win,)) / win
    smooth = np.empty_like(seq)
    for c in range(4):
        smooth[:, c] = np.convolve(padseq[:, c], kernel, mode='valid')
    return smooth

def crop_one(mp4_path: Path, pkl_path: Path, out_path: Path, target=88):
    lmks = pickle.load(open(pkl_path, "rb"))  # list of (68,2) or None
    cap  = cv2.VideoCapture(str(mp4_path))
    fps  = cap.get(cv2.CAP_PROP_FPS) or 25.0
    T    = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # xywh per frame (NaN on misses)
    xywh = []
    for i in range(T):
        pts = lmks[i] if i < len(lmks) else None
        if pts is None:
            xywh.append([np.nan]*4)
        else:
            xywh.append(tighten_xywh_from_mouth(pts))
    xywh = smooth_xywh(np.array(xywh, float), win=SMOOTH_WIN)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    wtr = cv2.VideoWriter(str(out_path), fourcc, fps, (target, target), isColor=False)
    ok_frames = 0

    for i in range(T):
        ok, frame = cap.read()
        if not ok: break
        x, y, w, h = xywh[i]
        x0 = max(0, int(round(x))); y0 = max(0, int(round(y)))
        x1 = min(frame.shape[1], int(round(x+w)))
        y1 = min(frame.shape[0], int(round(y+h)))
        crop = frame[y0:y1, x0:x1]
        if crop.size == 0:
            crop = frame  # fallback
        gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        lip  = cv2.resize(gray, (target, target), interpolation=cv2.INTER_AREA)
        wtr.write(lip); ok_frames += 1

    wtr.release(); cap.release()
    return ok_frames

# Batch over all MP4s in the angle folder
mp4s = sorted(VID25_DIR.glob("*.mp4"))
print(f"Found {len(mp4s)} videos in {VID25_DIR}")
for mp4 in tqdm(mp4s, desc=f"Cropping -> {OUT_DIR.name}"):
    base = mp4.stem
    pkl  = LMK_DIR / f"{base}.pkl"
    out  = OUT_DIR / f"{base}.mp4"
    if not pkl.exists():
        print(f"[skip] no landmarks for {base}")
        continue
    out.parent.mkdir(parents=True, exist_ok=True)
    n = crop_one(mp4, pkl, out, target=TARGET_SZ)
    print(f"[ok] {base}: {n} frames -> {out}")
print("Done.")


Found 377 videos in C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25\straightcam


Cropping -> straightcam:   0%|          | 1/377 [00:01<10:41,  1.71s/it]

[ok] sa1: 163 frames -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25crop\straightcam\sa1.mp4


Cropping -> straightcam:   1%|          | 2/377 [00:02<08:02,  1.29s/it]

[ok] sa2: 133 frames -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25crop\straightcam\sa2.mp4


...

[ok] sx97: 141 frames -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25crop\straightcam\sx97.mp4


Cropping -> straightcam: 100%|█████████▉| 376/377 [07:32<00:00,  1.05it/s]

[ok] sx98: 116 frames -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25crop\straightcam\sx98.mp4


Cropping -> straightcam: 100%|██████████| 377/377 [07:33<00:00,  1.20s/it]

[ok] sx99: 140 frames -> C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\lipspeakers\Lipspkr1\Clips\processed\video25crop\straightcam\sx99.mp4
Done.


## Data Augmentation
This section outlines Data Augmentation Techniques created during this project. 
Different techniques will be outlined in more detail below

In [7]:
#point to your TCD_TIMIT root
from pathlib import Path
DATA_ROOT = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
SPEAKER_TYPE = ["lipspeakers", "volunteers"]
VOLUNTEER_NUM = ["01M","02M","03F","04F","05M","06F","07M","08F","09M","10F"]
CAM_ANGLE = ["straightcam", "30degcam"]
PROCESSED_DIR = DATA_ROOT / SPEAKER_TYPE[1] / VOLUNTEER_NUM[1] / "Clips" / "processed"

## Interpolation 
Currently this idea entails taking the 30fps video and interpolating to a higher fps and then down sampling back to 30fps using these new frames
**Audio Note**
Audio needs to be realigned with the audio
Audio can also lead or lag by a small amount
ffmpeg used for interpolation

In [8]:
# @title Convert the cropped 25fps video to 50fps using FFmpeg's minterpolate filter
import subprocess, shutil
from pathlib import Path

# === EDIT THESE ===
VID_IN  = PROCESSED_DIR / "video25crop" / CAM_ANGLE[0] / "sa1.mp4"  # input 25fps
VID_OUT = PROCESSED_DIR / "interpolation" / "video50crop" / CAM_ANGLE[0] / "sa1_50fps.mp4"

# Ensure output folder exists
OUT_PATH = Path(VID_OUT)
DATA_PATH = Path(VID_IN)

assert shutil.which("ffmpeg"), "FFmpeg not found on PATH."
assert DATA_PATH.exists(), f"Input not found: {DATA_PATH}"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# Interpolate to 50 fps (one new frame between each original)
cmd = [
    "ffmpeg",
    "-i", str(DATA_PATH),
    "-vf", 'minterpolate=mi_mode=mci:mc_mode=aobmc:me_mode=bidir:fps=50',
    "-c:v", "libx264", "-crf", "0", "-preset", "veryslow", "-pix_fmt", "yuv420p",
    "-c:a", "copy",  # keep original audio untouched
    str(OUT_PATH),
]
subprocess.run(cmd, check=True)
print("Done:", OUT_PATH)


Done: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\02M\Clips\processed\interpolation\video50crop\straightcam\sa1_50fps.mp4


In [ ]:
import subprocess
from pathlib import Path

# Example explicit paths (or reuse your PROCESSED_DIR/CAM_ANGLE)
VID_IN        = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\02M\Clips\processed\interpolation\video50crop\straightcam\sa1_50fps.mp4")
BASE_OUT_ROOT = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\02M\Clips\processed")
CAMERA_ANGLE  = "straightcam"   # or "30degcam"

def _phase_label_ms(phase: float) -> str:
    ms_rounded = round(phase * 1000.0, 2)
    return f"{str(ms_rounded).replace('.', 'p')}ms"

def downsample_50_to_25(
    vid_in: Path,
    base_out_root: Path,
    camera_angle: str,
    phases = (1/50,),                 # default: only the interpolated frames
    round_type = ("near",),           # "near" is fine here
    sync_audio: bool = False          # False = keep audio (introduce +20ms AV offset)
):
    """
    Writes to:
      base_out_root / "interpolation" / "video25crop_interpolate" / f"phase_{LABEL}" / camera_angle / <file>.mp4
    """
    written = []
    for p in phases:
        phase_dir = base_out_root / "interpolation" / "video25crop_interpolate" / f"phase_{_phase_label_ms(p)}" / camera_angle
        phase_dir.mkdir(parents=True, exist_ok=True)

        for r in round_type:
            out_path = phase_dir / f"{vid_in.stem}_25fps_{r}.mp4"

            # Pick frames at 25fps starting at phase 'p' seconds (e.g., 0.02 for interpolated-only)
            # Reset PTS so the timeline starts at 0 after applying the phase.
            vf = f"fps=fps=25:start_time={p}:round={r},setpts=PTS-{p}/TB"

            # Audio:
            # - sync_audio=False: leave audio untouched -> constant +p video lead (e.g., +20ms)
            # - sync_audio=True: advance audio by 'p' to re-sync with the chosen frames
            af = None
            if sync_audio:
                # Advance audio by p (may drop the first p of audio if present)
                af = f"asetpts=PTS-{p}/TB"

            cmd = ["ffmpeg","-y","-i", str(vid_in), "-vf", vf]
            if af:
                cmd += ["-af", af]
            cmd += [
                "-c:v", "libx264", "-crf", "18", "-preset", "veryfast", "-pix_fmt", "yuv420p",
                "-c:a", "aac", "-b:a", "128k",
                str(out_path),
            ]

            subprocess.run(cmd, check=True)
            print("Wrote:", out_path)
            written.append(out_path)
    return written



# (B) Interpolated-only frames, SYNC audio (no AV offset; audio advanced by 20ms)
outputs_synced = downsample_50_to_25(
    vid_in=VID_IN,
    base_out_root=BASE_OUT_ROOT,
    camera_angle=CAMERA_ANGLE,
    phases=(1/50,),
    round_type=("near",),
    sync_audio=True
)
 

Wrote: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\02M\Clips\processed\interpolation\video25crop_interpolate\phase_20p0ms\straightcam\sa1_50fps_25fps_near.mp4
Wrote: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\02M\Clips\processed\interpolation\video25crop_interpolate\phase_20p0ms\straightcam\sa1_50fps_25fps_near.mp4


In [ ]:
# test downsample fucntions on output of upsample 60fps video
#"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\02M\Clips\processed\interpolation\video60crop\straightcam\sa1_60fps.mp4"
import cv2
cap = cv2.VideoCapture(str(VID_IN))




## 4. Build file/label lists and count frames

In [ ]:
def write_lists_and_counts(spk: str, ids: list):
    spk_root = ROOT / "volunteers" / spk
    clips = spk_root / "Clips"
    video_dir = clips / "video"
    audio_dir = clips / "audio"

    file_list   = clips / "file.list"
    label_list  = clips / "label.list"
    nfa_file    = clips / "nframes.audio"
    nfv_file    = clips / "nframes.video"

    lines_f = []
    lines_l = []
    lines_nfa = []
    lines_nfv = []

    for cid in tqdm(ids, desc=f"[{spk}] Counting/Listing", ncols=100):
        fid = f"trainval/{cid}"
        v = video_dir / f"{cid}.mp4"
        a = audio_dir / f"{cid}.wav"
        if not v.exists() or not a.exists():
            continue

        lines_f.append(fid+"\n")
        label = (ROOT / "volunteers" / spk / "Clips" / TXT_SUB / f"{cid}.txt").read_text(encoding="utf-8", errors="ignore").strip().lower()
        lines_l.append(label+"\n")

        nfa = count_audio_samples(a)
        nfv = count_video_frames(v)
        lines_nfa.append(str(nfa)+"\n")
        lines_nfv.append(str(nfv)+"\n")

    file_list.write_text("".join(lines_f), encoding="utf-8")
    label_list.write_text("".join(lines_l), encoding="utf-8")
    nfa_file.write_text("".join(lines_nfa), encoding="utf-8")
    nfv_file.write_text("".join(lines_nfv), encoding="utf-8")
    return clips

for spk, ids in ALL_IDS.items():
    write_lists_and_counts(spk, ids)

print("Wrote file.list, label.list, nframes.audio, nframes.video.")

## 5. Train SentencePiece vocab (with automatic fallback)

In [ ]:
import sentencepiece as spm
import shutil

def train_sentencepiece_for_speaker(spk: str, vocab_size: int = 1000):
    clips = ROOT / "volunteers" / spk / "Clips"
    label_list = clips / "label.list"
    out_dir = clips / f"spm{vocab_size}"
    out_dir.mkdir(exist_ok=True, parents=True)
    prefix = out_dir / f"spm_unigram{vocab_size}"

    txt = label_list.read_text(encoding="utf-8").lower()
    tmp = (clips / "labels_tmp.txt")
    tmp.write_text(txt, encoding="utf-8")

    size = vocab_size
    while size >= 128:
        try:
            spm.SentencePieceTrainer.Train(
                input=tmp.as_posix(),
                model_prefix=prefix.as_posix(),
                model_type="unigram",
                vocab_size=size,
                character_coverage=1.0,
                unk_id=3, bos_id=0, eos_id=2, pad_id=1,
                num_threads=2,
            )
            model_txt = str(prefix) + ".txt"
            dict_txt = clips / "dict.wrd.txt"
            shutil.copyfile(model_txt, dict_txt)
            print(f"[info] SentencePiece trained at size {size}. Saved dict to {dict_txt}")
            try: tmp.unlink()
            except Exception: pass
            return dict_txt
        except Exception as e:
            print(f"[info] SP fallback: {e} → trying smaller size")
            size = int(size * 0.8)
    raise RuntimeError("SentencePiece training failed; corpus likely too small.")

DICT_PATHS = {}
for spk in SPEAKERS:
    DICT_PATHS[spk] = train_sentencepiece_for_speaker(spk)

print("SPM done.")

## 6. Split data into train/valid/test and write AV‑HuBERT manifests

In [ ]:
def auto_split_ids(ids: list, pin_valid=set(["sa1","sa2"]), valid_ratio=0.1, test_ratio=0.05):
    ids = sorted(ids)
    valid = [i for i in ids if i in pin_valid]
    rest = [i for i in ids if i not in pin_valid]
    n_valid = max(len(valid), int(round(len(ids)*valid_ratio)))
    n_test  = int(round(len(ids)*test_ratio))
    if len(valid) < n_valid:
        extra = rest[:(n_valid - len(valid))]
        valid = valid + extra
        rest  = rest[(n_valid - len(valid)):] if (n_valid - len(valid))>0 else rest
    test = rest[:n_test]
    train = [i for i in ids if i not in set(valid) | set(test)]
    return train, valid, test

def write_tsv_and_wrd(spk: str, train_ids, valid_ids, test_ids):
    spk_root = ROOT / "volunteers" / spk
    clips = spk_root / "Clips"
    video_dir = clips / "video"
    audio_dir = clips / "audio"

    data_dir = clips / "433h_data"
    data_dir.mkdir(exist_ok=True, parents=True)

    nfa = [int(x.strip()) for x in (clips / "nframes.audio").read_text().splitlines()]
    nfv = [int(x.strip()) for x in (clips / "nframes.video").read_text().splitlines()]
    fids = [x.strip() for x in (clips / "file.list").read_text().splitlines()]
    labels = [x.strip() for x in (clips / "label.list").read_text().splitlines()]
    assert len(nfa) == len(nfv) == len(fids) == len(labels)

    rows = []
    for fid, lab, a, v in zip(fids, labels, nfa, nfv):
        cid = fid.split("/")[-1]
        rows.append((cid, lab, a, v))

    split_map = {"train": set(train_ids), "valid": set(valid_ids), "test": set(test_ids)}
    buckets = {"train": [], "valid": [], "test": []}

    for cid, lab, a, v in rows:
        rel_id = cid
        vpath = (video_dir / f"{cid}.mp4").absolute().as_posix()
        apath = (audio_dir / f"{cid}.wav").absolute().as_posix()
        for split in ["train", "valid", "test"]:
            if cid in split_map[split]:
                buckets[split].append((rel_id, lab, a, v, vpath, apath))
                break

    for split, items in buckets.items():
        with open(data_dir / f"{split}.tsv", "w", encoding="utf-8") as fo:
            fo.write("/\n")
            for rel_id, lab, a, v, vpath, apath in items:
                fo.write("\t".join([rel_id, vpath, apath, str(v), str(a)]) + "\n")
        with open(data_dir / f"{split}.wrd", "w", encoding="utf-8") as fo:
            for _, lab, *_ in items:
                fo.write(lab + "\n")

    dict_src = clips / "dict.wrd.txt"
    import shutil
    shutil.copyfile(dict_src, data_dir / "dict.wrd.txt")

    for split in ["train", "valid", "test"]:
        wrd_path = data_dir / f"{split}.wrd"
        ltr_path = data_dir / f"{split}.ltr"
        with open(wrd_path, "r", encoding="utf-8") as fi, open(ltr_path, "w", encoding="utf-8") as fo:
            for line in fi:
                sent = line.strip().replace(" ", "|")
                letters = " ".join(list(sent))
                fo.write(letters + "\n")

    return data_dir

SPLITS = {}
for spk, ids in ALL_IDS.items():
    if "{VALID_SPLIT_MODE}" == "file" and {bool(False)}:
        valid_ids = [x.strip() for x in Path(VALID_IDS_FILE).read_text().splitlines() if x.strip()]
        rest = [i for i in ids if i not in set(valid_ids)]
        test_n = max(1, int(round(len(ids)*TEST_RATIO)))
        test_ids = rest[:test_n]
        train_ids = [i for i in ids if i not in set(valid_ids)|set(test_ids)]
    else:
        train_ids, valid_ids, test_ids = auto_split_ids(ids, pin_valid=set(list(PIN_VALID)), valid_ratio=VALID_RATIO, test_ratio=TEST_RATIO)

    data_dir = write_tsv_and_wrd(spk, train_ids, valid_ids, test_ids)
    SPLITS[spk] = dict(train=len(train_ids), valid=len(valid_ids), test=len(test_ids), data_dir=str(data_dir))

SPLITS

## 7. Quick QA

In [ ]:
from pprint import pprint
pprint(SPLITS)

for spk in SPEAKERS:
    data_dir = Path(SPLITS[spk]["data_dir"])
    print(f"\n--- {spk} :: {data_dir} ---")
    for fn in ["train.tsv", "valid.tsv", "test.tsv", "train.wrd"]:
        p = data_dir / fn
        if p.exists():
            print(f"\n[{fn} head]")
            print("\n".join(p.read_text().splitlines()[:6]))

## 8. (Optional) Data augmentation — placeholders

In [ ]:


# paths
inp = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\01M\Clips\sa1_60fps.mp4")
out = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\01M\Clips\probe_frame.png")
out.parent.mkdir(parents=True, exist_ok=True)

# locate ffmpeg (or hardcode r"C:\ffmpeg\bin\ffmpeg.exe")
ffmpeg = shutil.which("ffmpeg")
assert ffmpeg, "FFmpeg not found on PATH."

# exact command translated to Python:
cmd = [
    ffmpeg, "-y",
    "-i", str(inp),
    "-vf", r"select=gte(n\,50)",  # note the backslash before the comma
    "-vframes", "1",
    str(out),
]

# run it
subprocess.run(cmd, check=True)
print(f"Wrote: {out}")


Wrote: C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT\volunteers\01M\Clips\probe_frame.png


## 9. Summary & next steps

- Converted videos to **25 fps**, audios to **16 kHz mono**, and cropped mouth ROI to **112×112**.
- Wrote lists: `file.list`, `label.list`, `nframes.audio`, `nframes.video`.
- Trained SentencePiece (with auto‑fallback) and saved `dict.wrd.txt`.
- Created AV‑HuBERT manifests under `Clips/433h_data/` for each speaker.

Now upload `Clips/433h_data` into your training environment and use the clean training notebook we made.

_Generated on 2025-10-09 11:41 UTC._

In [ ]:
from pathlib import Path

# Your existing roots
DATA_ROOT = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")
SPEAKER   = "02M"  # you can loop later if you want

# Where your 25fps full-face videos and 16k audio live
VID25_DIR = DATA_ROOT / rf"volunteers\{SPEAKER}\Clips\processed\video25\{CAM_ANGLE}"   # 25 fps MP4s (full face)
AUDIO_DIR = DATA_ROOT / rf"volunteers\{SPEAKER}\Clips\processed\audio16k\{CAM_ANGLE}"
TEXT_DIR  = DATA_ROOT / rf"volunteers\{SPEAKER}\Clips\text\{CAM_ANGLE}"

# Landmark outputs and ROI outputs
LMK_DIR   = DATA_ROOT / "landmarks"                          # .pkl files will mirror relative paths
ROI88_DIR = DATA_ROOT / rf"volunteers\{SPEAKER}\Clips\video\straightcam_roi88"  # 88×88 mp4s
ROI88_DIR.mkdir(parents=True, exist_ok=True)

# dlib model files (already downloaded)
DLIB_CNN = r"C:\models\dlib\mmod_human_face_detector.dat"
DLIB_L68 = r"C:\models\dlib\shape_predictor_68_face_landmarks.dat"

# AV-HuBERT repo path (where detect_landmark.py is)
AVH_REPO = Path(r"C:\path\to\av_hubert")  # <- set this to your local clone
DETECT_PY = AVH_REPO / r"avhubert\preparation\detect_landmark.py"


In [ ]:
from pathlib import Path

# Root of TCD-TIMIT on your machine
DATA_ROOT = Path(r"C:\Users\irish\Computer_Electronic_Engineering_Year5\AVSR_project\Data\TCD_TIMIT")

# Which speakers & cameras to process
SPEAKERS = ["02M"]                     # e.g. ["01M","02M","03F", ...]
CAMS     = ["straightcam", "30degcam"] # run one or both

# Output geometry / rates
TARGET_FPS = 25
TARGET_SR  = 16_000
ROI_SIZE   = 88
MARGIN     = 0.35   # mouth box expansion; tweak 0.25–0.45 if needed
